In [1]:
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [3]:
x = random((30,2))
y = np.dot(x, [2., 3.]) + 1.

In [4]:
x[:5]

array([[ 0.512 ,  0.1683],
       [ 0.3381,  0.0375],
       [ 0.1718,  0.777 ],
       [ 0.2601,  0.9465],
       [ 0.8984,  0.5386]])

In [5]:
y[:5]

array([ 2.529 ,  1.7887,  3.6747,  4.3596,  4.4124])

In [6]:
lm = Sequential([ Dense(1, input_shape=(2,)) ])
lm.compile(optimizer=SGD(lr=0.1), loss='mse')

In [7]:
lm.evaluate(x, y, verbose=0)

7.8706235885620117

In [8]:
lm.fit(x, y, nb_epoch=5, batch_size=1)

Epoch 1/5
30/30 [==============================] - 0s - loss: 0.5454     
Epoch 2/5
30/30 [==============================] - 0s - loss: 0.0469     
Epoch 3/5
30/30 [==============================] - 0s - loss: 0.0262     
Epoch 4/5
30/30 [==============================] - 0s - loss: 0.0165     
Epoch 5/5
30/30 [==============================] - 0s - loss: 0.0100         


In [9]:
lm.evaluate(x, y, verbose=0)

0.0061074690893292427

In [10]:
lm.get_weights()

[array([[ 1.7897],
        [ 2.7692]], dtype=float32), array([ 1.2297], dtype=float32)]

In [11]:
sample = True
if sample:
    path = "data_sample/"
else: 
    path = "data/"
    
model_path = 'models/'

if not os.path.exists(model_path): os.mkdir(model_path)
if not os.path.exists(path): os.mkdir(path)